In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/data/covid/acute_dyspnea_data/2020_0701/diagnosis-codes.csv')

In [ ]:
df

In [4]:
df_covid = df[df['TermCodeMapped'] == 'U07.1']
df_covid = df_covid.drop_duplicates('hosp_id')

In [ ]:
df_covid

In [6]:
df_covid.to_csv('/data/covid/model_ICU_care/data/2020_0701_covid_hospid_icd.csv', index=False)

## Check against labs

In [7]:
df_labs = pd.read_csv('/data/covid/acute_dyspnea_data/2020_0701/labs.csv')

In [ ]:
df_labs

In [9]:
df_lab_covid = df_labs[df_labs['result_name'].str.contains('COVID')]

In [10]:
df_lab_covid[['result_termid', 'result_name']].drop_duplicates().sort_values(by=['result_termid'])

,result_termid,result_name
1639682,703605,COVID
1639684,703606,Novel Coronavirus (COVID-19) Result:
1637898,703699,COVID Comment
1637897,703725,"ER Novel Coronavirus (COVID-19), PCR"
4313861,703786,COVID Interpretation


In [11]:
df_lab_covid[['result_termid', 'result_name', 'value']].sort_values(by=['result_termid', 'value']).drop_duplicates(subset=['result_termid', 'result_name']).iloc[[1,3]]

,result_termid,result_name,value
1990285,703606,Novel Coronavirus (COVID-19) Result:,Detected
1647162,703725,"ER Novel Coronavirus (COVID-19), PCR",Detected


In [ ]:
df_lab_covid

In [13]:
# Checks if everyone with a "COVID comment" or "COVID" (img links) also has the main COVID lab result(s)
set(df_lab_covid[df_lab_covid['result_termid'].isin([703605, 703699])]['hosp_id'].unique()) <= set(df_lab_covid[~df_lab_covid['result_termid'].isin([703605, 703699])]['hosp_id'].unique())

True

In [14]:
uniq_vals = {}
for termid in [703606, 703725]:
    records = df_lab_covid.query('result_termid == @termid')
    uniq_vals[termid] = sorted(records['value'].str.strip().unique())

In [15]:
uniq_vals

{703606: ['Detected', 'Invalid', 'Not detected'],
 703725: ['Detected', 'Invalid', 'Not detected']}

In [16]:
df_covid = df_labs[df_labs['result_termid'].isin([703606, 703725])].sort_values(by=['hosp_id', 'result_termid', 'value'])

In [17]:
df_covid['value'] = df_covid['value'].str.strip()

In [ ]:
df_covid

In [19]:
df_final = df_covid.drop_duplicates(subset=['hosp_id', 'result_termid'], keep='first')

In [ ]:
df_final

In [29]:
df_final.to_csv('/data/covid/model_ICU_care/data/2020_0701_covid_hospid_labs_tested.csv')

In [21]:
df_pos = df_final[df_final['value'] == 'Detected'].drop_duplicates(subset=['hosp_id'])

In [ ]:
df_pos

In [28]:
df_pos.to_csv('/data/covid/model_ICU_care/data/2020_0701_covid_hospid_labs_pos.csv')

In [24]:
# Relies on this sorted order to drop records with a priority Detected > everything else
df_final = df_covid[['hosp_id', 'result_termid', 'value']].drop_duplicates(subset=['hosp_id', 'result_termid'], keep='first')
df_final = df_final.set_index(['hosp_id', 'result_termid']).unstack()
df_final.columns = df_final.columns.droplevel(0)

In [25]:
df_flag = (df_final == 'Detected').fillna(False).astype(int)
# df_flag = df_flag.sum(axis=1).rename('COVID_flag').to_frame()

In [ ]:
df_flag

In [27]:
df_flag.max(axis=1).sum()

385